In [1]:
%matplotlib inline

# Imports

In [2]:
import os
import pandas as pd
from collections import defaultdict

from meta import config, config_tickers
# from meta.env_portfolio_allocation.env_portfolio_yahoofinance import StockPortfolioEnv
from meta.env_portfolio_allocation.env_portfolio_yahoofinance import StockPortfolioEnv
from meta.data_processor import DataProcessor
import numpy as np
import cvxpy as cp
import seaborn as sns
import matplotlib.pyplot as plt

from agents.stablebaselines3_models import DRLAgent
from stable_baselines3 import A2C


In [17]:
# function from example notebooks
def data_split(df, start, end, target_date_col="time"):
    """
    split the dataset into training or testing using time
    :param data: (df) pandas dataframe, start, end
    :return: (df) pandas dataframe
    """
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    dates = pd.to_datetime(df[target_date_col])
    data = df[(dates >= start) & (dates < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data


In [4]:
# example markowitz agent using info dict from environment to do optimization at each time step
class MarkowitzAgent:
    """Provides implementations for Markowitz agent (mean-variance optimization)
    Attributes
    ----------
        env: gym environment class
            user-defined class
    Methods
    -------
        prediction()
            make a prediction in a test dataset and get result history
    """

    def __init__(
            self,
            env,
            risk_aversion=10,
            annual_risk_free_rate=0.03 # disregard risk free rate since RL disregards
            ):
        super().__init__()
        self.risk_aversion = risk_aversion
        self.env = env
        # compute daily risk free rate from annual risk free rate
        # self.risk_free_rate = (1 + annual_risk_free_rate) ** (1 / 365) - 1
        # disable risk free rate for now
        self.risk_free_rate = -1

    def get_model(self, model_name, model_kwargs):
        raise NotImplementedError()

    def train_model(self, model, cwd, total_timesteps=5000):
        raise NotImplementedError()

    def act(self, state, info):
        """
        This is the core of markowitz portfolio optimization
        it maximizes the éxpected_return - risk_aversion * risk
        with expected_return = mean_returns @ portfolio_weights
        and risk = portfolio_weights.T @ cov @ portfolio_weights
        The constraints say that the weights must be positive and sum to 1

        returns the action as the weights of the portfolio
        """
        # unpack state to get covariance and means
        data = info["data"].copy()
        # from the data estimate returns and covariances
        cov = data.iloc[-1].cov_list
        mean_returns = data[
            data["time"] == data["time"].max()
        ]["ewm_returns"].to_numpy()

        # solve markowitz model with cvxpy
        # initialize model
        num_stocks = len(mean_returns)
        weights = cp.Variable(num_stocks)
        risk_free_weight = cp.Variable(1)
        # define constraints
        # constraints = [cp.sum(weights) + risk_free_weight ==
        #                1, weights >= 0, risk_free_weight >= 0]
        constraints = [cp.sum(weights) == 1,
                       weights >= 0,
                    #    risk_free_weight >= 0
                       ]
        # define objective
        portfolio_return = mean_returns @ weights # + risk_free_weight*self.risk_free_rate
        portfolio_risk = cp.quad_form(weights, cov)
        # define objective
        objective = cp.Maximize(
            portfolio_return - self.risk_aversion * portfolio_risk)
        # define problem
        problem = cp.Problem(objective, constraints)
        # solve problem
        problem.solve()
        # get weights
        weights = weights.value
        # get action. if using risk free rate then integrate it into the action
        action = weights
        # action = np.concatenate([weights, risk_free_weight.value])
        action = np.maximum(action, 0)
        action = action / np.sum(action)
        return action

    def prediction(self, environment):
        # args = Arguments(env=environment)
        # args.if_off_policy
        # args.env = environment

        # test on the testing env
        state, info = environment.reset()
        day = environment.sorted_times[environment.time_index]
        history = defaultdict(list)

        total_asset = environment.portfolio_value
        history["date"].append(day)
        history["total_assets"].append(total_asset)
        history["episode_return"].append(0)
        # episode_total_assets.append(environment.initial_amount)
        done = False
        while not done:
            action = self.act(state, info)
            state, reward, done, trunc, info = environment.step(action)
            day = environment.sorted_times[environment.time_index]


            total_asset = environment.portfolio_value
            episode_return = total_asset / environment.initial_amount
            history["date"].append(day)
            history["total_assets"].append(total_asset)
            history["episode_return"].append(episode_return)
        print("Test Finished!")
        # return episode total_assets on testing data
        print("episode_return", episode_return)
        return pd.DataFrame(history)

In [5]:
# some hyperparameters for finrl
start_date=config.TRAIN_START_DATE
end_date=config.TRADE_END_DATE
ticker_list=config_tickers.DOW_30_TICKER
time_interval='1D'
data_source='yahoofinance'
technical_indicator_list= config.INDICATORS
if_vix=True
hmax=100
initial_amount=1000000
transaction_cost_pct=0.001
reward_scaling=1e-4
use_cached_model = False


# Download and process data

In [6]:
dp = DataProcessor(
    data_source=data_source,
    start_date=start_date,
    end_date=end_date,
    time_interval=time_interval,
)

price_array, tech_array, turbulence_array = dp.run(
    ticker_list, technical_indicator_list, if_vix=if_vix, cache=True, select_stockstats_talib=0
)

# add covariance matrix as states
df = dp.dataframe
df = df.sort_values(['time', 'tic'], ignore_index=True)
df.index = df.time.factorize()[0]

df["pct_change"] = df.groupby("tic").close.pct_change()

cov_list = []
# look back is one year
lookback = 252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i, :]
    price_lookback = data_lookback.pivot_table(
        index='time', columns='tic', values='close')
    return_lookback = price_lookback.pct_change().dropna()
    covs = return_lookback.cov().values
    cov_list.append(covs)
df["mean_pct_change_lookback"] = df.rolling(lookback)["pct_change"].mean()
df["ewm_returns"] = df["pct_change"].ewm(span=50).mean()
df_cov = pd.DataFrame(
    {'time': df.time.unique()[lookback:], 'cov_list': cov_list})
df = df.merge(df_cov, on='time')
df = df.sort_values(['time', 'tic']).reset_index(drop=True)


yahoofinance successfully connected
Using cached file ./cache/AXP_AMGN_AAPL_BA_CAT_CSCO_CVX_GS_HD_HON_IBM_INTC_JNJ_KO_JPM_MCD_MMM_MRK_MSFT_NKE_PG_TRV_UNH_CRM_VZ_V_WBA_WMT_DIS_DOW_yahoofinance_2014-01-01_2021-12-01_1D.pickle
tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
[*********************100%***********************]  1 of 1 completed
           date       open       high        low      close  adjusted_close  \
0    2014-01-02  14.320000  14.590000  14.000000  14.230000       14.230000   
1    2014-01-03  14.060000  14.220000  13.570000  13.760000       13.760000   
2    2014-01-06  13.410000  14.000000  13.220000  13.550000       13.550000   
3    2014-01-07  12.380000  13.280000  12.160000  12.920000       12.920000  

/home/aaron/Research/finrl_experiments/FinRL-Meta/meta/data_processors/yahoofinance.py:177: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(tmp_df)


Data clean for ^VIX is finished.
Data clean all finished!
Successfully transformed into array


# Run markowitz model on test data

In [7]:
test_df = data_split(
    df,
    start=config.TEST_START_DATE,
    end=config.TEST_END_DATE
)

stock_dimension = len(test_df.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": hmax,
    "initial_amount": initial_amount,
    "transaction_cost_pct": transaction_cost_pct,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": reward_scaling
}
e_test_gym = StockPortfolioEnv(df=test_df, **env_kwargs)
agent = MarkowitzAgent(e_test_gym)
markowitz_history_df = agent.prediction(e_test_gym)
markowitz_history_df["method"] = "markowitz"

Stock Dimension: 30, State Space: 30
begin_total_asset:1000000
end_total_asset:1317065.746292424
Sharpe:  1.8108090198148892
Test Finished!
episode_return 1.317065746292424


# Train RL model and evaluate on test data

In [8]:

train_df = data_split(df, start=config.TRAIN_START_DATE,
                        end=config.TRAIN_END_DATE)
stock_dimension = len(train_df.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": hmax,
    "initial_amount": initial_amount,
    "transaction_cost_pct": transaction_cost_pct,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": reward_scaling
}
# use cached model if you are iterating on evaluation code
if use_cached_model and os.path.exists("saved_models/a2c_model.pt"):
    trained_a2c = A2C.load("saved_models/a2c_model.pt")
else:
    e_train_gym = StockPortfolioEnv(df=train_df, **env_kwargs)
    agent = DRLAgent(env=e_train_gym)

    A2C_PARAMS = {
        "n_steps": 10,
        "ent_coef": 0.005,
        "learning_rate": 0.0004
    }
    model_a2c = agent.get_model(model_name="a2c", model_kwargs=A2C_PARAMS)
    trained_a2c = agent.train_model(
        model=model_a2c,
        tb_log_name='a2c',
        total_timesteps=40000
        )
    # save trained_a2c model
    trained_a2c.save("saved_models/a2c_model.pt")

# evaluate on test data
test_df = data_split(
    df,
    start=config.TEST_START_DATE,
    end=config.TEST_END_DATE
)
e_test_gym = StockPortfolioEnv(df=test_df, **env_kwargs)

df_daily_return, df_actions = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment=e_test_gym,
)
df_daily_return["method"] = "a2c"


Stock Dimension: 30, State Space: 30
{'n_steps': 10, 'ent_coef': 0.005, 'learning_rate': 0.0004}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to tensorboard_log/a2c/a2c_4
begin_total_asset:1000000
end_total_asset:1306448.6670203465
Sharpe:  2.0796516678287724
begin_total_asset:1000000
end_total_asset:1307639.1310502
Sharpe:  2.0902243062155734
begin_total_asset:1000000
end_total_asset:1285445.4166034753
Sharpe:  1.9961449627713799
begin_total_asset:1000000
end_total_asset:1319296.7307926817
Sharpe:  2.202596064213726
begin_total_asset:1000000
end_total_asset:1272895.9003085298
Sharpe:  1.9221607834792764
begin_total_asset:1000000
end_total_asset:1294929.4760228002
Sharpe:  2.089197367892366
begin_total_asset:1000000
end_total_asset:1317139.4855363015
Sharpe:  2.127869745726194
begin_total_asset:1000000
end_total_asset:1299975.4845626622
Sharpe:  2.0500233307988585
begin_total_asset:1000000
end_total_asset:1283107.5427514156
Shar

begin_total_asset:1000000
end_total_asset:1287833.8194915457
Sharpe:  1.9689713852336732
begin_total_asset:1000000
end_total_asset:1263509.5362592814
Sharpe:  1.838719071195812
begin_total_asset:1000000
end_total_asset:1313967.7778771343
Sharpe:  2.0907619818487384
begin_total_asset:1000000
end_total_asset:1277117.6526189276
Sharpe:  1.930226673775957
begin_total_asset:1000000
end_total_asset:1287393.601063561
Sharpe:  1.9720365363551648
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 93       |
|    ep_rew_mean        | 1.14e+08 |
| time/                 |          |
|    fps                | 201      |
|    iterations         | 400      |
|    time_elapsed       | 19       |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -42.4    |
|    explained_variance | 1.79e-07 |
|    learning_rate      | 0.0004   |
|    n_updates          | 399      |
|    policy_loss        | 2.87e+08 |
|  

begin_total_asset:1000000
end_total_asset:1305350.290238444
Sharpe:  2.092551065926783
begin_total_asset:1000000
end_total_asset:1279610.4512576032
Sharpe:  1.9692362838560564
begin_total_asset:1000000
end_total_asset:1275247.3636130237
Sharpe:  1.905340497714614
begin_total_asset:1000000
end_total_asset:1302227.201979787
Sharpe:  2.040455926675958
begin_total_asset:1000000
end_total_asset:1300586.2561145285
Sharpe:  2.0558133148359987
begin_total_asset:1000000
end_total_asset:1290031.20545703
Sharpe:  2.000471395841479
begin_total_asset:1000000
end_total_asset:1271826.8687750082
Sharpe:  1.9117467962356882
begin_total_asset:1000000
end_total_asset:1283795.046594785
Sharpe:  1.9405508469138868
begin_total_asset:1000000
end_total_asset:1282874.0184216963
Sharpe:  1.9993673247550168
begin_total_asset:1000000
end_total_asset:1238581.2513159541
Sharpe:  1.7048046506660486
begin_total_asset:1000000
end_total_asset:1277470.1401245196
Sharpe:  1.9201657425224454
------------------------------

begin_total_asset:1000000
end_total_asset:1282367.6642587823
Sharpe:  1.9478265669896695
begin_total_asset:1000000
end_total_asset:1279441.392857954
Sharpe:  1.9526698383899812
begin_total_asset:1000000
end_total_asset:1285818.2742083487
Sharpe:  1.9380224142287412
begin_total_asset:1000000
end_total_asset:1275430.8234627682
Sharpe:  1.937824349364703
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.13e+08  |
| time/                 |           |
|    fps                | 204       |
|    iterations         | 1100      |
|    time_elapsed       | 53        |
|    total_timesteps    | 11000     |
| train/                |           |
|    entropy_loss       | -42.3     |
|    explained_variance | 0         |
|    learning_rate      | 0.0004    |
|    n_updates          | 1099      |
|    policy_loss        | 2.56e+08  |
|    reward             | 1195520.1 |
|    std                | 0.993     |


begin_total_asset:1000000
end_total_asset:1253263.3030467394
Sharpe:  1.79377964548967
begin_total_asset:1000000
end_total_asset:1271748.6039599099
Sharpe:  1.9095793887543686
begin_total_asset:1000000
end_total_asset:1310411.1787571318
Sharpe:  2.0946806348906555
begin_total_asset:1000000
end_total_asset:1274138.8117308256
Sharpe:  1.9217926832910364
begin_total_asset:1000000
end_total_asset:1299565.2197648603
Sharpe:  2.0412564255997028
begin_total_asset:1000000
end_total_asset:1276768.6906522296
Sharpe:  1.9258219792349658
begin_total_asset:1000000
end_total_asset:1284640.8316165726
Sharpe:  1.9840364594217021
begin_total_asset:1000000
end_total_asset:1283528.919830087
Sharpe:  1.9560810809401865
begin_total_asset:1000000
end_total_asset:1279082.549177439
Sharpe:  1.925280340943537
begin_total_asset:1000000
end_total_asset:1249945.7390226151
Sharpe:  1.7536587882147605
begin_total_asset:1000000
end_total_asset:1254812.5743158367
Sharpe:  1.7833410278774833
--------------------------

...

begin_total_asset:1000000
end_total_asset:1252898.9891214876
Sharpe:  1.822292884884775
begin_total_asset:1000000
end_total_asset:1290734.4906448135
Sharpe:  2.048078695155222
begin_total_asset:1000000
end_total_asset:1316763.469112589
Sharpe:  2.1538233534448596
begin_total_asset:1000000
end_total_asset:1254556.1511515907
Sharpe:  1.792388612384324
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.12e+08  |
| time/                 |           |
|    fps                | 203       |
|    iterations         | 2500      |
|    time_elapsed       | 122       |
|    total_timesteps    | 25000     |
| train/                |           |
|    entropy_loss       | -42.2     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0004    |
|    n_updates          | 2499      |
|    policy_loss        | 2.96e+08  |
|    reward             | 1244146.2 |
|    std                | 0.989     |
| 

begin_total_asset:1000000
end_total_asset:1286483.3823772576
Sharpe:  2.005824663836732
begin_total_asset:1000000
end_total_asset:1268156.9125064292
Sharpe:  1.8653997273918348
begin_total_asset:1000000
end_total_asset:1273146.303072798
Sharpe:  1.9497007500631542
begin_total_asset:1000000
end_total_asset:1242569.3851059997
Sharpe:  1.7434773747768548
begin_total_asset:1000000
end_total_asset:1275043.9783173078
Sharpe:  1.9136623877938972
begin_total_asset:1000000
end_total_asset:1295577.7103250707
Sharpe:  2.0273263996073556
begin_total_asset:1000000
end_total_asset:1278443.6024948207
Sharpe:  1.9079228987663832
begin_total_asset:1000000
end_total_asset:1266922.386993531
Sharpe:  1.8822088593362702
begin_total_asset:1000000
end_total_asset:1249451.5740508684
Sharpe:  1.796346811128382
begin_total_asset:1000000
end_total_asset:1245389.3115175366
Sharpe:  1.7480336147162916
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |


begin_total_asset:1000000
end_total_asset:1254373.4341537654
Sharpe:  1.8299574727652501
begin_total_asset:1000000
end_total_asset:1306752.0316384174
Sharpe:  2.0726104904934104
begin_total_asset:1000000
end_total_asset:1292395.6920684401
Sharpe:  2.00079157920154
begin_total_asset:1000000
end_total_asset:1251624.5847621607
Sharpe:  1.8162563119485133
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.13e+08  |
| time/                 |           |
|    fps                | 203       |
|    iterations         | 3200      |
|    time_elapsed       | 157       |
|    total_timesteps    | 32000     |
| train/                |           |
|    entropy_loss       | -42.1     |
|    explained_variance | 1.79e-07  |
|    learning_rate      | 0.0004    |
|    n_updates          | 3199      |
|    policy_loss        | 2.86e+08  |
|    reward             | 1082398.5 |
|    std                | 0.986     |


begin_total_asset:1000000
end_total_asset:1278477.0659422958
Sharpe:  1.9343110978038658
begin_total_asset:1000000
end_total_asset:1277726.8657414592
Sharpe:  1.9201743027676537
begin_total_asset:1000000
end_total_asset:1299253.1935428393
Sharpe:  2.04172790014739
begin_total_asset:1000000
end_total_asset:1256066.215092193
Sharpe:  1.8237081111931486
begin_total_asset:1000000
end_total_asset:1284770.2601909786
Sharpe:  1.9329962647040158
begin_total_asset:1000000
end_total_asset:1265289.0592914498
Sharpe:  1.8506775609797836
begin_total_asset:1000000
end_total_asset:1272438.3368820897
Sharpe:  1.8993761518469066
begin_total_asset:1000000
end_total_asset:1253810.992259301
Sharpe:  1.762335388646208
begin_total_asset:1000000
end_total_asset:1305026.758230713
Sharpe:  2.093726179249428
begin_total_asset:1000000
end_total_asset:1272662.7187512068
Sharpe:  1.8950184647318438
begin_total_asset:1000000
end_total_asset:1248928.9412151356
Sharpe:  1.7738540696019647
----------------------------

begin_total_asset:1000000
end_total_asset:1279416.9763617225
Sharpe:  1.9435994923339506
begin_total_asset:1000000
end_total_asset:1257679.5411187143
Sharpe:  1.858087338530323
begin_total_asset:1000000
end_total_asset:1283065.37088706
Sharpe:  1.9691666060084794
begin_total_asset:1000000
end_total_asset:1285098.9918940782
Sharpe:  1.9418479961412143
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 93        |
|    ep_rew_mean        | 1.13e+08  |
| time/                 |           |
|    fps                | 203       |
|    iterations         | 3900      |
|    time_elapsed       | 191       |
|    total_timesteps    | 39000     |
| train/                |           |
|    entropy_loss       | -42.1     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0004    |
|    n_updates          | 3899      |
|    policy_loss        | 2.65e+08  |
|    reward             | 1163179.4 |
|    std                | 0.984     |
|

In [9]:
df_daily_return["total_assets"] = (1+df_daily_return["daily_return"]).cumprod() * e_test_gym.initial_amount

# Plot returns comparison

In [10]:
cols = ["date", "total_assets", "method"]
df_to_plot = pd.concat([df_daily_return[cols], markowitz_history_df[cols]])

sns.lineplot(
    x="date",
    y="total_assets",
    hue="method",
    data=df_to_plot
    )
plt.title("return comparison")
plt.show()

# Get returns tearsheet using pyfolio
note that this requires a modified version of pyfolio which has bugs, ignore the download_data error as long as you get 
"yahoofinance successfully connected"

In [11]:
# code for analyzing returns using pyfolio, creating a tear sheet
import pyfolio as pf

In [13]:
try:
    _ = dp.processor.download_data(["^GSPC"], save_path="./data/sp500.csv")
except:
    pass

[*********************100%***********************]  1 of 1 completed
           date         open         high          low        close  \
0    2014-01-02  1845.859985  1845.859985  1827.739990  1831.979980   
1    2014-01-03  1833.209961  1838.239990  1829.130005  1831.369995   
2    2014-01-06  1832.310059  1837.160034  1823.729980  1826.770020   
3    2014-01-07  1828.709961  1840.099976  1828.709961  1837.880005   
4    2014-01-08  1837.900024  1840.020020  1831.400024  1837.489990   
...         ...          ...          ...          ...          ...   
1988 2021-11-23  4678.479980  4699.390137  4652.660156  4690.700195   
1989 2021-11-24  4675.779785  4702.870117  4659.890137  4701.459961   
1990 2021-11-26  4664.629883  4664.629883  4585.430176  4594.620117   
1991 2021-11-29  4628.750000  4672.950195  4625.259766  4655.270020   
1992 2021-11-30  4640.250000  4646.020020  4560.000000  4567.000000   

      adjusted_close      volume    tic  day  
0        1831.979980  308060000

In [14]:
sp500_data = pd.read_csv("./data/sp500.csv")

In [15]:
sp500_data = sp500_data.set_index(pd.to_datetime(sp500_data["date"]))
sp500_data.index = sp500_data.index.tz_localize('America/New_York')
sp500_data['pct_change'] = sp500_data.close.pct_change()
sp500_data = sp500_data.dropna()
# sp500_data.index = sp500_data.index.to_pydatetime()

# plot quantstats metrics
ideally we want to fix quantstats or compute the metrics ourselves, but this is just an example

quantstats is broken for me requiring the try, catch

In [16]:
# quantstats tear sheet
for method_name in df_to_plot.method.unique():
    print(f"{method_name} tear sheet")
    port_returns = df_to_plot[df_to_plot["method"] == method_name].drop_duplicates("date").set_index("date")["total_assets"]
    port_returns.index = port_returns.index.tz_localize('America/New_York')

    try:
        pf.create_returns_tear_sheet(port_returns.pct_change().dropna(), benchmark_rets=sp500_data["pct_change"])
    except:
        pass

a2c tear sheet
Entire data start date: 2020-08-04
Entire data end date: 2021-09-30
Backtest months: 13


/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/timeseries.py:718: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/plotting.py:693: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


,Backtest
Annual return,26.0%
Cumulative returns,30.8%
Annual volatility,14.3%
Sharpe ratio,1.69
Calmar ratio,3.09
Stability,0.89
Max drawdown,-8.4%
Omega ratio,1.33
Sortino ratio,2.56
Skew,0.02


markowitz tear sheet
Entire data start date: 2020-08-04
Entire data end date: 2021-09-30
Backtest months: 13


/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/timeseries.py:718: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/home/aaron/miniconda3/envs/finrl/lib/python3.8/site-packages/pyfolio/plotting.py:693: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


,Backtest
Annual return,26.7%
Cumulative returns,31.7%
Annual volatility,13.6%
Sharpe ratio,1.81
Calmar ratio,4.62
Stability,0.82
Max drawdown,-5.8%
Omega ratio,1.36
Sortino ratio,2.93
Skew,0.54
